<a href="https://colab.research.google.com/github/shanilev123456/Final-Project-ds/blob/master/data_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [236]:
from google.colab import files
uploaded = files.upload()

Saving rmi_results.csv to rmi_results (16).csv


In [237]:
import pandas as pd
df = pd.read_csv('rmi_results.csv')
df.head()

,Tender Number,Region,City,Purpose,Publication date,Status,Opening date,Submission deadline,Committee date,Compound Number,...,Winner Name,Final Price,Development Cost,Valuation Price,Minimum Price,Area m2,Plan,Parcel,Block,Section
0,250/2024,ירושלים,אבן ספיר,בנייה נמוכה/צמודת קרקע,02/09/2024,נדון בוועדת מכרזים,29/10/2024,10/03/2025 12:00,10/03/2025,א,...,גילי זקן,2396416.0,583193.0,2552814.0,1161530.0,507.0,מי/838,234,29951.0,67.0
1,250/2024,ירושלים,אבן ספיר,בנייה נמוכה/צמודת קרקע,02/09/2024,נדון בוועדת מכרזים,29/10/2024,10/03/2025 12:00,10/03/2025,ב,...,"נזיה שבלי, איהאם שבלי",2664000.0,583193.0,2681019.0,1219864.0,500.0,מי/838,239,29951.0,72.0
2,250/2024,ירושלים,אבן ספיר,בנייה נמוכה/צמודת קרקע,02/09/2024,נדון בוועדת מכרזים,29/10/2024,10/03/2025 12:00,10/03/2025,ג,...,"שלמה לב, מיכל לב",2181000.0,583193.0,2441703.0,1110975.0,490.0,מי/838,260,31396.0,180.0
3,361/2024,צפון,"כפר הורדים , שלב ג",בנייה נמוכה/צמודת קרקע -לייזמים/קבלנים,14/10/2024,נדון בוועדת מכרזים,23/12/2024,03/03/2025 14:00,03/03/2025,176,...,"יעד נדל""ן בגליל בע""מ",5000555.0,13175258.0,10073093.0,NaN,17527.0,ג/15442,183,21126.0,179.0
4,442/2024,מרכז,"כפר סבא , כפר סבא",בנייה נמוכה/צמודת קרקע,09/12/2024,נדון בוועדת מכרזים,30/12/2024,24/02/2025 12:00,25/02/2025,68ב,...,"רועי ציון בן דוד, רוית בן דוד",2711770.0,8286.0,3040000.0,1383200.0,404.0,כס/15/1,68ב,7593.0,65.0


In [238]:

df = df.drop(columns=['Purpose', 'Status', 'Winner Name', 'Submission deadline', 'Plan'])

In [239]:

print("missing data:")
print(df.isnull().sum())



df = df[df['Final Price'].notnull()]
df = df[df['Final Price'] != "אין הצעות למתחם זה"]


df['Units'] = df['Units'].fillna(df['Units'].median())
df['Valuation Price'] = df['Valuation Price'].fillna(df['Valuation Price'].median())
df['Minimum Price'] = df['Minimum Price'].fillna(df['Minimum Price'].median())
df['Development Cost'] = df['Development Cost'].fillna(df['Development Cost'].mean())

df = df[df['Compound Number'].notnull()]
print(f"missing data Compound Number : {len(df)}")


# עמודות גיאוגרפיות - מילוי בערך ברירת מחדל
df['Parcel'] = df['Parcel'].fillna("לא קיים")
df['Block'] = df['Block'].fillna("לא קיים")
df['Section'] = df['Section'].fillna("לא קיים")

print("\nmissing data after:")
print(df.isnull().sum())
print(f"missing data len: {len(df)}")

missing data:
Tender Number         0
Region                0
City                  0
Publication date      0
Opening date          0
Committee date        0
Compound Number       8
Units                58
Final Price         722
Development Cost    342
Valuation Price     255
Minimum Price       499
Area m2               3
Parcel              367
Block               103
Section             103
dtype: int64
missing data Compound Number : 2272

missing data after:
Tender Number       0
Region              0
City                0
Publication date    0
Opening date        0
Committee date      0
Compound Number     0
Units               0
Final Price         0
Development Cost    0
Valuation Price     0
Minimum Price       0
Area m2             0
Parcel              0
Block               0
Section             0
dtype: int64
missing data len: 2272


In [240]:

df['Final Price'] = df['Final Price'].replace(',', '', regex=True).astype(float)
df['Area m2'] = df['Area m2'].replace(',', '', regex=True).astype(float)
print(f"before filter Final Price: {len(df)}")
# גבולות ידניים נוספים
lower_bound = df['Final Price'].quantile(0.05)
upper_bound = df['Final Price'].quantile(0.95)
df = df[(df['Final Price'] >= lower_bound) & (df['Final Price'] <= upper_bound)]
print(f"after filter Final Price: {len(df)}")
df = df[(df['Area m2'] >= 50) & (df['Area m2'] <= 50000)]
print(f"after filter Area m2: {len(df)}")

before filter Final Price: 2272
after filter Final Price: 2045
after filter Area m2: 2045


In [241]:

duplicate_rows = df.duplicated(subset=['Tender Number', 'Compound Number'], keep=False)
print(f"len of doubles row: {duplicate_rows.sum()}")

df = df.drop_duplicates(subset=['Tender Number', 'Compound Number'], keep='first')
print(f"after: {len(df)}")

len of doubles row: 0
after: 2045


In [242]:

df['Publication date'] = pd.to_datetime(df['Publication date'], errors='coerce')
df['Opening date'] = pd.to_datetime(df['Opening date'], errors='coerce')
df['Committee date'] = pd.to_datetime(df['Committee date'], errors='coerce')


df['Publication_year'] = df['Publication date'].dt.year
df['Publication_month'] = df['Publication date'].dt.month
df['Publication_day'] = df['Publication date'].dt.day

df['Opening_year'] = df['Opening date'].dt.year
df['Opening_month'] = df['Opening date'].dt.month
df['Opening_day'] = df['Opening date'].dt.day

df['Committee_year'] = df['Committee date'].dt.year
df['Committee_month'] = df['Committee date'].dt.month
df['Committee_day'] = df['Committee date'].dt.day


df['Days_Publication_to_Opening'] = (df['Opening date'] - df['Publication date']).dt.days
df['Days_Publication_to_Committee'] = (df['Committee date'] - df['Publication date']).dt.days
df['Days_Opening_to_Committee'] = (df['Committee date'] - df['Opening date']).dt.days


df['Days_Publication_to_Opening'] = df['Days_Publication_to_Opening'].fillna(df['Days_Publication_to_Opening'].median())
df['Days_Publication_to_Committee'] = df['Days_Publication_to_Committee'].fillna(df['Days_Publication_to_Committee'].median())
df['Days_Opening_to_Committee'] = df['Days_Opening_to_Committee'].fillna(df['Days_Opening_to_Committee'].median())


for col in ['Publication_year', 'Publication_month', 'Publication_day',
            'Opening_year', 'Opening_month', 'Opening_day',
            'Committee_year', 'Committee_month', 'Committee_day']:
    df[col] = df[col].fillna(df[col].median())


df = df.drop(columns=['Publication date', 'Opening date', 'Committee date'])


print(df.head())

  Tender Number   Region                 City Compound Number  Units  \
0      250/2024  ירושלים             אבן ספיר               א    1.0   
1      250/2024  ירושלים             אבן ספיר               ב    1.0   
2      250/2024  ירושלים             אבן ספיר               ג    1.0   
4      442/2024     מרכז    כפר סבא , כפר סבא             68ב    1.0   
5      355/2024     דרום  נתיבות , מעלות הנחל             323    1.0   

   Final Price  Development Cost  Valuation Price  Minimum Price  Area m2  \
0    2396416.0          583193.0        2552814.0      1161530.0    507.0   
1    2664000.0          583193.0        2681019.0      1219864.0    500.0   
2    2181000.0          583193.0        2441703.0      1110975.0    490.0   
4    2711770.0            8286.0        3040000.0      1383200.0    404.0   
5     405550.0          492749.0         432234.0        55110.0    388.0   

   ... Publication_day Opening_year Opening_month  Opening_day  \
0  ...             9.0         2024   

<ipython-input-242-69884f13699a>:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Opening date'] = pd.to_datetime(df['Opening date'], errors='coerce')


In [243]:
print(df['Final Price'].describe())

count    2.045000e+03
mean     1.096435e+06
std      9.033169e+05
min      7.000000e+04
25%      3.903910e+05
50%      8.283000e+05
75%      1.521000e+06
max      4.200770e+06
Name: Final Price, dtype: float64


In [244]:
df['Price per m2'] = df['Final Price'] / df['Area m2']

numeric_columns = [
    'Final Price', 'Development Cost', 'Valuation Price', 'Minimum Price',
    'Area m2', 'Units',
    'Publication_year', 'Publication_month', 'Publication_day',
    'Opening_year', 'Opening_month', 'Opening_day',
    'Committee_year', 'Committee_month', 'Committee_day',
    'Days_Publication_to_Opening', 'Days_Publication_to_Committee', 'Days_Opening_to_Committee','Price per m2'
]

for col in numeric_columns:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())


print(df.head())

  Tender Number   Region                 City Compound Number  Units  \
0      250/2024  ירושלים             אבן ספיר               א    0.0   
1      250/2024  ירושלים             אבן ספיר               ב    0.0   
2      250/2024  ירושלים             אבן ספיר               ג    0.0   
4      442/2024     מרכז    כפר סבא , כפר סבא             68ב    0.0   
5      355/2024     דרום  נתיבות , מעלות הנחל             323    0.0   

   Final Price  Development Cost  Valuation Price  Minimum Price   Area m2  \
0     0.563192          0.033136         0.160387       0.300402  0.015870   
1     0.627970          0.033136         0.168517       0.315576  0.015616   
2     0.511043          0.033136         0.153341       0.287251  0.015253   
4     0.639535          0.000099         0.191280       0.358063  0.012130   
5     0.081232          0.027939         0.025918       0.012597  0.011549   

   ... Opening_year Opening_month Opening_day  Committee_year  \
0  ...          1.0      0.818182

In [245]:
print(df['Final Price'].describe())

count    2045.000000
mean        0.248485
std         0.218680
min         0.000000
25%         0.077562
50%         0.183574
75%         0.351266
max         1.000000
Name: Final Price, dtype: float64


In [246]:

df = pd.get_dummies(df, columns=['Region', 'City'], prefix=['Region', 'City'])


target_encode_columns = ['Block', 'Parcel', 'Section']

for col in target_encode_columns:
    target_mean = df.groupby(col)['Final Price'].mean()
    df[col] = df[col].map(target_mean)

for col in target_encode_columns:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

In [247]:
import pandas as pd


city_columns = [col for col in df.columns if col.startswith('City_')]


cities = set()
for col in city_columns:
    city_name = col.replace('City_', '').split(',')[0].strip()
    cities.add(city_name)


for city in cities:
    city_specific_columns = [col for col in city_columns if col.startswith(f'City_{city}')]
    df[f'City_{city}'] = df[city_specific_columns].any(axis=1)


city_avg_price_per_m2 = {}
for city in cities:
    avg_price = df[df[f'City_{city}']]['Price per m2'].mean()
    city_avg_price_per_m2[city] = avg_price


df['City_Avg_PricePerM2'] = 0.0
for city in cities:
    df.loc[df[f'City_{city}'], 'City_Avg_PricePerM2'] = city_avg_price_per_m2[city]


<ipython-input-247-f9c2e74ebf42>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'City_{city}'] = df[city_specific_columns].any(axis=1)
<ipython-input-247-f9c2e74ebf42>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'City_{city}'] = df[city_specific_columns].any(axis=1)
<ipython-input-247-f9c2e74ebf42>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

In [248]:
df.to_csv('cleaned_data.csv', index=False, encoding='utf-8-sig')
files.download('cleaned_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>